<a href="https://colab.research.google.com/github/CAB314/2048game/blob/main/game_ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model_basic


In [ ]:
!mkdir /content/game

code = ['import numpy as np\nfrom IPython.display import clear_output\n', 'class Model():\n', '  def __init__(self, size = (4, 4), score = 0, done = False):\n', '    self._size = size\n', '    self._score = score\n', '    self._done = done\n', '    self._empty = 1\n', '    self._tiles = np.array([[self._empty for c in range(self._size[1])] for r in range(self._size[0])])\n', '\n', '  def __str__(self):\n', '    return "Score:{}\\nGameover:{}\\n______________________\\n{}".format(self._score, self._done, self._tiles)\n', '\n', '  def action(self, command):\n', '    param = {\n', '        "u":[range(self._size[0]), range(self._size[1]), -1, 0],\n', '        "d":[range(self._size[0] -1, -1, -1), range(self._size[1] -1, -1, -1), 1, 0],\n', '        "l":[range(self._size[0]), range(self._size[1]), 0, -1],\n', '        "r":[range(self._size[0] -1, -1, -1), range(self._size[1] -1, -1, -1), 0, 1]\n', '    }\n', '    rows, cols, dr, dc = param[command]\n', '    for row in rows:\n', '      for col in cols:\n', '        r = row\n', '        c = col\n', '\n', '        while True:\n', '          r += dr\n', '          c += dc\n', '          if 0 <= r < self._size[0] and 0 <= c < self._size[1]:\n', '            if self._tiles[r, c] == self._empty:\n', '              self._tiles[r, c] = self._tiles[r - dr, c - dc]\n', '              self._tiles[r - dr, c - dc] = self._empty\n', '\n', '            elif self._tiles[r, c] == self._tiles[r - dr, c - dc]:\n', '              self._tiles[r, c] += self._tiles[r - dr, c - dc]\n', '              self._tiles[r - dr, c - dc] = self._empty\n', '              self._score += self._tiles[r, c]\n', '              break\n', '            \n', '            else:\n', '              break\n', '\n', '          else:\n', '            break\n', '\n', '  def update(self):\n', '    rows, cols = np.where(self._tiles == self._empty)\n', '    if len(rows) == 0:\n', '      self._done = True\n', '\n', '    else:\n', '      idx = np.random.randint(0, len(rows))\n', '      row, col = rows[idx], cols[idx]\n', '      val = self.new_tile_val()\n', '      self._tiles[row, col] = val\n', '\n', '  def new_tile_val(self):\n', '    return 2\n', '  \n']

with open('/content/game/model.py', 'w') as f:
  for wr in code:
    print(wr, file=f, end = "")

In [ ]:
import sys
sys.path.append("/content/game")
from model import Model
import numpy as np
from IPython.display import clear_output, HTML, display, update_display
from google.colab import output

In [ ]:
class View():
  def __init__(self, model):
    self._model = model
    self._html = ""
    self._css = ""
    self._js = ""

  def css(self):
    self._css = """
    * {text-align:center;}
    .tile {font-size:24px;} 
    """

  def js(self):
    # update に引数を渡す
    self._js="""
    document.querySelector('#u').onclick = () => {google.colab.kernel.invokeFunction('update', ['u'], {});};
    document.querySelector('#d').onclick = () => {google.colab.kernel.invokeFunction('update', ['d'], {});};
    document.querySelector('#l').onclick = () => {google.colab.kernel.invokeFunction('update', ['l'], {});};
    document.querySelector('#r').onclick = () => {google.colab.kernel.invokeFunction('update', ['r'], {});};
    """

  def html(self):
    self.css()
    self.js()
    table = "<table border = '1', align = 'center'>"
    for r in range(self._model._size[0]):
      table += "<tr>"
      for c in range(self._model._size[1]):
        if(self._model._tiles[r, c] != self._model._empty):
          color = "ffcc{:02x}".format(min(np.log2(self._model._tiles[r, c]).astype("int") * 64, 255))
          table += "<td width = '60pt' height = '60pt' bgcolor = '{}'>{}</td>".format(color, self._model._tiles[r, c])
        else:
          table += "<td width = '60pt' height = '60pt'></td>"


      table += "</tr>"
    table += "</table>"

    self._html = """
    <html>
      <head>
        <style>{}</style>
        <script>{}</script>
      </head>
      <body>
        <h1>2048</h1>
        <hr>
        <h2>Score:{}</h2>
        <h2>Gameover:{}</h2>
        <hr>
        <button id = 'u'>↑</button>
        <br>
        <button id = 'l'>←</button>
        <button id = 'd'>↓</button>
        <button id = 'r'>→</button>
        <hr>
        <div class = 'tiles'>{}</div>
      </body>
      """.format(self._css, self._js, self._model._score, self._model._done, table)
  
  def display(self):
    self.html()
    display(HTML(self._html))
    output.register_callback('update', self.update)

  def update(self, command):
    self._model.action(command)
    self._model.update()
    self.html()
    clear_output()
    self.display()



In [ ]:
def main(m = Model()):
  v = View(model = m)
  m.update()
  v.display()

if __name__ =="__main__":
  main()

# Expand

In [ ]:
class View_with_end(View):
  def __init__(self, model):
    self._model = model
    self._html = ""
    self._css = ""
    self._js = ""

  def css(self):
    self._css = """
    * {text-align:center;}
    .tile {font-size:24px;} 
    """

  def js(self):
    # update に引数を渡す
    self._js="""
    document.querySelector('#u').onclick = () => {google.colab.kernel.invokeFunction('update', ['u'], {});};
    document.querySelector('#d').onclick = () => {google.colab.kernel.invokeFunction('update', ['d'], {});};
    document.querySelector('#l').onclick = () => {google.colab.kernel.invokeFunction('update', ['l'], {});};
    document.querySelector('#r').onclick = () => {google.colab.kernel.invokeFunction('update', ['r'], {});};
    """

  def html(self):
    self.css()
    self.js()
    table = "<table border = '1', align = 'center'>"
    for r in range(self._model._size[0]):
      table += "<tr>"
      for c in range(self._model._size[1]):
        if(self._model._tiles[r, c] != self._model._empty):
          color = "ffcc{:02x}".format(min(np.log2(self._model._tiles[r, c]).astype("int") * 64, 255))
          table += "<td width = '60pt' height = '60pt' bgcolor = '{}'>{}</td>".format(color, self._model._tiles[r, c])
        else:
          table += "<td width = '60pt' height = '60pt'></td>"


      table += "</tr>"
    table += "</table>"

    self._html = """
    <html>
      <head>
        <style>{}</style>
        <script>{}</script>
      </head>
      <body>
        <h1>2048</h1>
        <hr>
        <h2>Score:{}</h2>
        <h2>Gameover:{}</h2>
        <hr>
        <button id = 'u'>↑</button>
        <br>
        <button id = 'l'>←</button>
        <button id = 'd'>↓</button>
        <button id = 'r'>→</button>
        <hr>
        <div class = 'tiles'>{}</div>
      </body>
      """.format(self._css, self._js, self._model._score, self._model._done, table)

    if(self._model._done == True):
      self._html = """
      <html>
        <head>
          <style>{}</style>
        </head>
        <body>
          <h1>2048</h1>
          <hr>
          <h2>Score:{}</h2>
          <h2>Gameover:{}</h2>
          <hr>
          <hr>
          <div class = 'tiles'>{}</div>
        </body>
        """.format(self._css, self._model._score, self._model._done, table)
  



In [ ]:
def main_with_end(m = Model()):
  v = View_with_end(model = m)
  m.update()
  v.display()

In [ ]:
class Model2(Model):#with cost
  def __init__(self, size = (4, 4), score = 0, done = False):
    self._size = size
    self._score = score
    self._done = done
    self._empty = 1
    self._tiles = np.array([[self._empty for c in range(self._size[1])] for r in range(self._size[0])])


  def action(self, command):
    self._score -= 1
    param = {
        "u":[range(self._size[0]), range(self._size[1]), -1, 0],
        "d":[range(self._size[0] -1, -1, -1), range(self._size[1] -1, -1, -1), 1, 0],
        "l":[range(self._size[0]), range(self._size[1]), 0, -1],
        "r":[range(self._size[0] -1, -1, -1), range(self._size[1] -1, -1, -1), 0, 1]
    }
    rows, cols, dr, dc = param[command]
    for row in rows:
      for col in cols:
        r = row
        c = col

        while True:
          r += dr
          c += dc
          if 0 <= r < self._size[0] and 0 <= c < self._size[1]:
            if self._tiles[r, c] == self._empty:
              self._tiles[r, c] = self._tiles[r - dr, c - dc]
              self._tiles[r - dr, c - dc] = self._empty

            elif self._tiles[r, c] == self._tiles[r - dr, c - dc]:
              self._tiles[r, c] += self._tiles[r - dr, c - dc]
              self._tiles[r - dr, c - dc] = self._empty
              self._score += self._tiles[r, c]
              break
            
            else:
              break

          else:
            break

In [ ]:
if __name__ =="__main__":
  main_with_end(Model2(size=(1, 1)))

In [ ]:
class Model3(Model2):#cost, clear, limit
  def __init__(self, size = (4, 4), score = 0, done = False, clear = 2048, limit = -1):
    self._size = size
    self._score = score
    self._done = done
    self._empty = 1
    self._tiles = np.array([[self._empty for c in range(self._size[1])] for r in range(self._size[0])])
    self._clear = clear
    self._limit = limit
    self._reward = clear * 100

  def update(self):


    rows, cols = np.where(self._tiles == self._clear)
    self._limit -= 1

    if len(rows) != 0:
      self._done = True
      self._score += self._reward

    elif self._limit == -1:
      self._done = True
      self._score -= self._reward

    if(self._done != True):
      rows, cols = np.where(self._tiles == self._empty)

      if len(rows) == 0:
        self._done = True
        self._score -= self._reward

      else:
        idx = np.random.randint(0, len(rows))
        row, col = rows[idx], cols[idx]
        val = self.new_tile_val()
        self._tiles[row, col] = val





In [ ]:
if __name__ =="__main__":
  main_with_end(Model3(size = (4, 4), score = 0, done = False, clear = 16, limit = -1))